# Class Practice Session

## Goal:
- Create a scene with instances of multiple objects layed our on a floor

## Details
- Create a square floor object on the YZ plane whose center is at (0,0,0).
  - Manually create the vertices of the floor object with position and texture coordinates.
  - Create a bound object similar to the one returned by create3DAssimpObject, but with bounds relevant to the floor object. As mentioned earlier, this bound will be used to set our camera and light.
  - Write a shader program to draw this floor with floor-wood.jpg as the texture.
- Create a cell grid on the floor object using a uniform subdivision along each dimension, and compute a jittered center for each cell to place the instance object.
- Load the model object that you want to instantiate.
- Compute matrices to scale, random rotate, and translate model object(s) to the instance center. Let  us call them instanceMatices.
- Create a [Shader Storage Buffer Object (SSBO)](https://wikis.khronos.org/opengl/Shader_Storage_Buffer_Object) to store the matrices in GPU and bind it to a binding unit 0. Unlike the vertex array buffers, SSBOs have special layout qualifiers for specifying binding locations.
```
instanceBuffer = gl.buffer(<matrix float array data>)
instanceBuffer.bind_to_storage_buffer(<buffer binding location>)
```
If you are using glm to create the instance matrices, then you must make sure that the data in the matrix float array has the glm matrix in column order. One way to do it is:
  - use the create a list of matrices where each item is list(matrix)
  - convert to a numpy array and flatten it: numpy.array(<matrix list>).flatten()
- Update the model object vertex shader program to define the buffer binding location,
```
    layout(binding = <buffer binding location>, std430) readonly buffer InstanceData {
        mat4 instanceMatrix[];
    };
```  
and access the instantMatrix for every instance using gl_InstanceID, a predefined instance ID. Composite this instance matrix with model matrix
- make renderable render calls using the number of instances you have, ex: renderable.render(instances = \<number of instance\>)

## Imports

In [ ]:
import pygame
import moderngl
import numpy
import glm
from loadModelUsingAssimp_V3 import create3DAssimpObject

In [ ]:
size = 5 
floor_positions = size * numpy.array([
    [-0.5, 0, -0.5],
    [0.5, 0, -0.5],
    [0.5, 0, 0.5],

    [0.5, 0, 0.5],
    [-0.5, 0, 0.5],
    [-0.5, 0, -0.5]
]).astype("float32")

floor_uv = numpy.array([
    [0,0],
    [1,0],
    [1,1],

    [0,0],
    [0,1],
    [0,0]
])

floor_geom = numpy.concat((floor_positions, floor_uv), axis=1).flatten()
class bound:
    boundingBox = [size* glm.vec3([-0.5, 0, -0.5]), [size* glm.vec3([0.5, 0, 0.5])]]
    center = glm.vec3(0)
    radius = size * 1.732

rng = numpy.random.default_rng()

nSubDivisions = 10 
totalInstances = nSubDivisions * nSubDivisions
delta = size/nSubDivisions
centers = []
#jCenters = []
minP = bound.boundingBox[0]

for i in range(nSubDivisions):
    for j in range(nSubDivisions):
        dxdy = rng.uniform(0, delta/2, 2)
        #jCenter = minP + glm.vec3([i*delta+delta/2, 0, j*delta+delta/2]) <- right in the middle of the boxes 
        center = minP + glm.vec3([i*delta+delta/4+dxdy[0], 0, j*delta+delta/4+dxdy[1]])
        centers.append(center)
        #jCenters.append(jCenter)

objSize = delta * 0.75

In [ ]:
floor_geom

In [ ]:
bound.radius
vars(bound)

In [ ]:
from myPlot import plot 
plot(floor_positions, nSubDivisions, centers) #jCenters

In [ ]:
#
# Programs
#

floor_vertex_shader= '''
    #version 430 core
    layout (location=0) in vec3 position;
    layout (location=1) in vec2 uv;
        
    uniform mat4 view, perspective;
    
    out vec2 f_uv; // Texture coordinate
    out vec3 f_normal; // Normal vector in World Coordinates
    out vec3 f_position; // postion in world coordinates
    void main() {
        f_uv = uv;
        vec4 P = M*vec4(position, 1);
        f_position = P.xyz; 
        gl_Position = perspective*view*P;
        f_normal = vec3(1,0,1);
    }
    '''
floor_fragment_shader= '''
    #version 430 core
    in vec2 f_uv;
    in vec3 f_normal;
    // in vec3 f_position;
    
    uniform sampler2D map;
    uniform vec3 light;
    
    uniform float shininess;
    uniform vec3 eye_position;
    uniform vec3 k_diffuse;
    
    // Add output variable here
    out vec4 out_color;
    
    vec3 computeColor(){
        vec3 L = normalize(light.xyz);
        vec3 materialColor = texture(map, f_uv).rgb;
        vec3 N = normalize(f_normal);
        float NdotL = dot(N,L);
        vec3 color = vec3(0.);
        return color;
    }
    void main() {
        out_color = vec4(computeColor(), 1);
    }
    '''    

### ToDo: Create Square floor object on ZX plane and create bounds for the floor.
<ins>Note</ins>:  
The floor will be our main Scene/World object. Its extents define the scene extent. So its bounds will be used to create camera, light, and view transformations.

### ToDo: Generate points on the floor where you would like to place instances of the object 
For $jittering$ the points and for $random\ rotation$ of individual objects we will use the random number generator of numpy.  Here is how you can generate them.
- Create a random number generator: rng = numpy.random.default_rng()
- Generate one or more random numbers uniformly between low and high: rng.uniform(low, high) for a single random number and rng.uniform(low, high, size) for an array of uniform random numbers, where size is the length of the array.

### ToDo: Write shader code for the floor

### Read model

In [ ]:
model_file = "chair_table_class/scene.gltf"
modelObj = create3DAssimpObject(model_file)

### Write shader code for the Model Object

In [ ]:
#
# Programs
#

model_vertex_shader= '''
    #version 430 core
    layout (location=0) in vec3 position;
    layout (location=1) in vec3 normal;
    layout (location=2) in vec2 uv;
        
    uniform mat4 model, view, perspective;
    
    out vec2 f_uv; // Texture coordinate
    out vec3 f_normal; // Normal vector in World Coordinates
    out vec3 f_position; // postion in world coordinates
    void main() {
        mat4 M = model;
        f_uv = uv;
        vec4 P = M*vec4(position, 1);
        f_position = P.xyz;
        gl_Position = perspective*view*P;
        mat3 normalMatrix = mat3(transpose(inverse(M)));// inverse transpose of model transformation
        f_normal = normalize(normalMatrix*normal);
    }
    '''
model_fragment_shader= '''
    #version 430 core
    in vec2 f_uv;
    in vec3 f_normal;
    in vec3 f_position;
    
    uniform sampler2D map;
    uniform vec3 light;
    
    uniform float shininess;
    uniform vec3 eye_position;
    uniform vec3 k_diffuse;
    
    // Add output variable here
    out vec4 out_color;
    
    vec3 computeColor(){
        vec3 L = normalize(light.xyz);
        vec3 materialColor = texture(map, f_uv).rgb;
        vec3 N = normalize(f_normal);
        float NdotL = dot(N,L);
        vec3 color = vec3(0.);
        vec3 ambientColor = 0.1*materialColor;
        if (NdotL>0.){
            vec3 diffuselyReflectedColor = materialColor * NdotL;
            // Compute specular color
            vec3 V = normalize(eye_position - f_position);
            vec3 H = normalize(L+V);
            vec3 specularlyReflectedColor = vec3(0);
            if (shininess > 0)
                specularlyReflectedColor = vec3(pow(dot(N,H), shininess));
            color = k_diffuse * diffuselyReflectedColor + specularlyReflectedColor;
        }
        color += ambientColor; 
        return color;
    }
    void main() {
        out_color = vec4(computeColor(), 1);
    }
    '''    

### Define Camera Parameters

In [ ]:
#bound = modelObj.bound
width = 840
height = 480

displacement_vector = 2*bound.radius*glm.rotate(glm.vec3(0,1,0), glm.radians(60), glm.vec3(1,0,0)) #

light_displacement_vector = 2*bound.radius*glm.rotate(glm.vec3(0,1,0), glm.radians(45), glm.vec3(1,0,0)) 
    
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0,1,0)

### View volume parameters
fov_radian = glm.radians(30) # In radian
aspect = width/height
near = bound.radius
far = 3*bound.radius
perspectiveMatrix = glm.perspective(fov_radian, aspect, near, far)

## Intialize

#### Enable anti-aliasing in the GPU pipeline
 - At the time of context creation, request a multisample buffer.: pygame.display.gl_set_attribute(GL_MULTISAMPLEBUFFERS, 1).
 - Specify the number of samples per pixel for multisampling: pygame.display.gl_set_attribute(GL_MULTISAMPLESAMPLES, 16). The higher the number, the better the antialiasing. Check the maximum number of samples supported in your driver. (ctx.max_samples)

In [ ]:
pygame.init() # Initlizes its different modules. Display module is one of them.
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLEBUFFERS, 1)
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLESAMPLES, 16)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Class Practice: Instructor")
gl = moderngl.get_context() # Get Previously created context.
gl.info["GL_VERSION"]

## Create shader program(s), Renderables and Texture Samplers

In [ ]:
model_program = gl.program(model_vertex_shader, model_fragment_shader)
modelObj.createRenderableAndSampler(model_program)

#### ToDo: Create floor_program, Renderable and texture sampler
Recap: Renderables in modernGL are created using the following method:
```
ctx.vertex_array(floor_program,
                [(gl.buffer(floor_geom), "3f 2f", "position", "uv")],
            )
```
Texture samplers are created using:
- loading texture image: pygame.image.load(\<texture image file name\>)
- convert the loaded data to a byte array: pygame.image.tobytes(\<loaded texture image\>,"RGB", True) # True is to flip the Y axis of the image
- creating texture: ctx.texture(\loaded texture image\>.get_size(), data = \<byteArrayData\>, components=3)
- and then creating the texture sampler: ctx.sampler(texture=\<createD texture\>, filter=(ctx.LINEAR_MIPMAP_LINEAR, ctx.LINEAR), repeat_x = True, repeat_y = True)

### Render loop

In [ ]:
floor_renderable = gl.program(floor_vertex_shader, floor_fragment_shader)
ctx.vertex_array(floor_program,
                [(gl.buffer(floor_geom), "3f 2f", "position", "uv")],
            )
texture_img = pygame.image.load("floor-wood.jpg")
texture_data = pygame.image.tobytes(texture_image, "RGB", True)
ctx.texture()
ctx.sampler

In [ ]:
running = True
clock = pygame.time.Clock()
alpha = 0
pause = True
lightAngle = 0

gl.enable(gl.DEPTH_TEST)

while running:   
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type ==  pygame.KEYDOWN):
            if  event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
            elif event.key == pygame.K_LEFT:
                lightAngle -= 5
            elif event.key == pygame.K_RIGHT:
                lightAngle += 5
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspectiveMatrix = glm.perspective(fov_radian, width/height, near, far)

    # create the aspect ratio correction matrix
    new_displacement_vector = glm.rotate(displacement_vector, glm.radians(alpha), glm.vec3(0,1,0))

    new_light_displacement_vector = glm.rotate(light_displacement_vector, glm.radians(lightAngle), glm.vec3(0,1,0))
    
    eye_point = target_point + new_displacement_vector

    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    gl.clear(0.0, 0.0, 0.0)

    # ToDo: Render floor
    #    Set the view, perspective, light uniform values.Render the floor:
    #    attach the floor texture sampler to texture unit 0: <sampler>.use(0)
    #    set the map sampler value to 0
    #    make render call for the floor
    floor_renderable(render)
    # Render model
    program = floor_program
    program["view"].write(viewMatrix)
    program["perspective"].write(perspectiveMatrix)
    program["light"].write(new_light_displacement_vector)
    floor_sampler.use(0)
    program["map"]
    modelObj.render()
    
    pygame.display.flip()
    
    clock.tick(60)  # limits FPS to 10
    if not pause:
        alpha +=  1
        if alpha > 360:
            alpha = 0
    #running = False
pygame.display.quit()